In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option("display.max_columns",300)

# Questions

1. Can I predict income inequality in a county (measured by difference between top income quintile and bottom income quintile) based on factors like:
    - Educational attainment - 
    - Proportion of population major ethnic groups - 
    - Population density - *Master*
    - Birth rate - *all_data*
    - Unemployment rate - need to find
    - Target - diff between lowest and highest quintile https://factfinder.census.gov/faces/tableservices/jsf/pages/productview.xhtml?src=bkmk
        - this will be positive, bounded by zero. 
        - likely NA's for some low-pop counties
2. Does median education rate impact income inequality?
3. Does birth rate impact income inequality?

# Goals for 3/13

1. Procure all listed datapoints not already in the data folder
2. Initial EDA to determine distribution of data and appropriate treatment
3. Figure out how to use:
    - education 
        - just median # of year at school? 
        - distribution across buckets (no high school diploma, some college, college degree, graduate degree)
    - major ethnic groups
        - should do proportions of each within community?
        - is there a standard measure of diversity?

In [59]:
all_data = pd.read_csv('co-est2015-alldata-utf8-3142.csv')
acs_15 = pd.read_csv('ACS_15_5YR_S1901_with_ann_utf8.csv',header=[0,1])
geo = pd.read_csv('2015_demo_geo.csv')
master = pd.read_csv('2015_master_sd_rnd_nan_to_min.csv')

In [64]:
# all_data.info()
geo.shape
master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Data columns (total 21 columns):
Unnamed: 0                 3142 non-null int64
geo_id                     3142 non-null int64
state_name                 3142 non-null object
state_code                 3142 non-null object
county_name                3142 non-null object
area_sqmi                  3142 non-null float64
log_area_sqmi              3142 non-null float64
pop_est_2015               3142 non-null int64
log_pop_est_2015           3142 non-null float64
pop_den_2015               3142 non-null float64
log_pop_den_2015           3142 non-null float64
liquor_stores              3142 non-null float64
household_median_income    3142 non-null int64
hot_big                    3142 non-null float64
hot_small                  3142 non-null int64
bar_big                    3142 non-null float64
bar_small                  3142 non-null int64
hotels                     3142 non-null float64
bars                      

In [55]:
acs_15.head()

# Households; Estimate; Median income (dollars)
# Households; Margin of Error; Median income (dollars)



,GEO.id,GEO.id2,GEO.display-label,HC01_EST_VC01,HC01_MOE_VC01,HC02_EST_VC01,HC02_MOE_VC01,HC03_EST_VC01,HC03_MOE_VC01,HC04_EST_VC01,HC04_MOE_VC01,HC01_EST_VC02,HC01_MOE_VC02,HC02_EST_VC02,HC02_MOE_VC02,HC03_EST_VC02,HC03_MOE_VC02,HC04_EST_VC02,HC04_MOE_VC02,HC01_EST_VC03,HC01_MOE_VC03,HC02_EST_VC03,HC02_MOE_VC03,HC03_EST_VC03,HC03_MOE_VC03,HC04_EST_VC03,HC04_MOE_VC03,HC01_EST_VC04,HC01_MOE_VC04,HC02_EST_VC04,HC02_MOE_VC04,HC03_EST_VC04,HC03_MOE_VC04,HC04_EST_VC04,HC04_MOE_VC04,HC01_EST_VC05,HC01_MOE_VC05,HC02_EST_VC05,HC02_MOE_VC05,HC03_EST_VC05,HC03_MOE_VC05,HC04_EST_VC05,HC04_MOE_VC05,HC01_EST_VC06,HC01_MOE_VC06,HC02_EST_VC06,HC02_MOE_VC06,HC03_EST_VC06,HC03_MOE_VC06,HC04_EST_VC06,HC04_MOE_VC06,HC01_EST_VC07,HC01_MOE_VC07,HC02_EST_VC07,HC02_MOE_VC07,HC03_EST_VC07,HC03_MOE_VC07,HC04_EST_VC07,HC04_MOE_VC07,HC01_EST_VC08,HC01_MOE_VC08,HC02_EST_VC08,HC02_MOE_VC08,HC03_EST_VC08,HC03_MOE_VC08,HC04_EST_VC08,HC04_MOE_VC08,HC01_EST_VC09,HC01_MOE_VC09,HC02_EST_VC09,HC02_MOE_VC09,HC03_EST_VC09,HC03_MOE_VC09,HC04_EST_VC09,HC04_MOE_VC09,HC01_EST_VC10,HC01_MOE_VC10,HC02_EST_VC10,HC02_MOE_VC10,HC03_EST_VC10,HC03_MOE_VC10,HC04_EST_VC10,HC04_MOE_VC10,HC01_EST_VC11,HC01_MOE_VC11,HC02_EST_VC11,HC02_MOE_VC11,HC03_EST_VC11,HC03_MOE_VC11,HC04_EST_VC11,HC04_MOE_VC11,HC01_EST_VC13,HC01_MOE_VC13,HC02_EST_VC13,HC02_MOE_VC13,HC03_EST_VC13,HC03_MOE_VC13,HC04_EST_VC13,HC04_MOE_VC13,HC01_EST_VC15,HC01_MOE_VC15,HC02_EST_VC15,HC02_MOE_VC15,HC03_EST_VC15,HC03_MOE_VC15,HC04_EST_VC15,HC04_MOE_VC15,HC01_EST_VC18,HC01_MOE_VC18,HC02_EST_VC18,HC02_MOE_VC18,HC03_EST_VC18,HC03_MOE_VC18,HC04_EST_VC18,HC04_MOE_VC18,HC01_EST_VC19,HC01_MOE_VC19,HC02_EST_VC19,HC02_MOE_VC19,HC03_EST_VC19,HC03_MOE_VC19,HC04_EST_VC19,HC04_MOE_VC19,HC01_EST_VC20,HC01_MOE_VC20,HC02_EST_VC20,HC02_MOE_VC20,HC03_EST_VC20,HC03_MOE_VC20,HC04_EST_VC20,HC04_MOE_VC20
,Id,Id2,Geography,Households; Estimate; Total,Households; Margin of Error; Total,Families; Estimate; Total,Families; Margin of Error; Total,Married-couple families; Estimate; Total,Married-couple families; Margin of Error; Total,Nonfamily households; Estimate; Total,Nonfamily households; Margin of Error; Total,"Households; Estimate; Less than $10,000","Households; Margin of Error; Less than $10,000","Families; Estimate; Less than $10,000","Families; Margin of Error; Less than $10,000","Married-couple families; Estimate; Less than $10,000","Married-couple families; Margin of Error; Less than $10,000","Nonfamily households; Estimate; Less than $10,000","Nonfamily households; Margin of Error; Less than $10,000","Households; Estimate; $10,000 to $14,999","Households; Margin of Error; $10,000 to $14,999","Families; Estimate; $10,000 to $14,999","Families; Margin of Error; $10,000 to $14,999","Married-couple families; Estimate; $10,000 to $14,999","Married-couple families; Margin of Error; $10,000 to $14,999","Nonfamily households; Estimate; $10,000 to $14,999","Nonfamily households; Margin of Error; $10,000 to $14,999","Households; Estimate; $15,000 to $24,999","Households; Margin of Error; $15,000 to $24,999","Families; Estimate; $15,000 to $24,999","Families; Margin of Error; $15,000 to $24,999","Married-couple families; Estimate; $15,000 to $24,999","Married-couple families; Margin of Error; $15,000 to $24,999","Nonfamily households; Estimate; $15,000 to $24,999","Nonfamily households; Margin of Error; $15,000 to $24,999","Households; Estimate; $25,000 to $34,999","Households; Margin of Error; $25,000 to $34,999","Families; Estimate; $25,000 to $34,999","Families; Margin of Error; $25,000 to $34,999","Married-couple families; Estimate; $25,000 to $34,999","Married-couple families; Margin of Error; $25,000 to $34,999","Nonfamily households; Estimate; $25,000 to $34,999","Nonfamily households; Margin of Error; $25,000 to $34,999","Households; Estimate; $35,000 to $49,999","Households; Margin of Error; $35,000 to $49,999","Families; Estimate; $35,000 to $49,999","Families; Margin of Error; $35,000 to $49,999","Married-couple families; Estima

In [54]:
acs_15.columns.levels;

In [56]:
all_data.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,NPOPCHG_2010,NPOPCHG_2011,NPOPCHG_2012,NPOPCHG_2013,NPOPCHG_2014,NPOPCHG_2015,BIRTHS2010,BIRTHS2011,BIRTHS2012,BIRTHS2013,BIRTHS2014,BIRTHS2015,DEATHS2010,DEATHS2011,DEATHS2012,DEATHS2013,DEATHS2014,DEATHS2015,NATURALINC2010,NATURALINC2011,NATURALINC2012,NATURALINC2013,NATURALINC2014,NATURALINC2015,INTERNATIONALMIG2010,INTERNATIONALMIG2011,INTERNATIONALMIG2012,INTERNATIONALMIG2013,INTERNATIONALMIG2014,INTERNATIONALMIG2015,DOMESTICMIG2010,DOMESTICMIG2011,DOMESTICMIG2012,DOMESTICMIG2013,DOMESTICMIG2014,DOMESTICMIG2015,NETMIG2010,NETMIG2011,NETMIG2012,NETMIG2013,NETMIG2014,NETMIG2015,RESIDUAL2010,RESIDUAL2011,RESIDUAL2012,RESIDUAL2013,RESIDUAL2014,RESIDUAL2015,GQESTIMATESBASE2010,GQESTIMATES2010,GQESTIMATES2011,GQESTIMATES2012,GQESTIMATES2013,GQESTIMATES2014,GQESTIMATES2015,RBIRTH2011,RBIRTH2012,RBIRTH2013,RBIRTH2014,RBIRTH2015,RDEATH2011,RDEATH2012,RDEATH2013,RDEATH2014,RDEATH2015,RNATURALINC2011,RNATURALINC2012,RNATURALINC2013,RNATURALINC2014,RNATURALINC2015,RINTERNATIONALMIG2011,RINTERNATIONALMIG2012,RINTERNATIONALMIG2013,RINTERNATIONALMIG2014,RINTERNATIONALMIG2015,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,55253,55175,55038,55290,55347,89,593,-78,-137,252,57,151,636,615,574,623,600,152,507,558,583,504,467,-1,129,57,-9,119,133,33,20,16,16,18,19,49,398,-161,-166,125,-140,82,418,-145,-150,143,-121,8,46,10,22,-10,45,455,455,455,455,455,455,455,11.572789,11.138479,10.416194,11.293597,10.846281,9.225478,10.106133,10.579514,9.136393,8.442022,2.347311,1.032347,-0.163320,2.157204,2.404259,0.363924,0.289782,0.290347,0.326300,0.343466,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
1,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,186659,190396,195126,199713,203709,928,3466,3737,4730,4587,3996,517,2187,2092,2160,2186,2240,532,1825,1879,1902,2044,1992,-15,362,213,258,142,248,69,187,172,170,212,221,856,2743,3327,4211,3799,3469,925,2930,3499,4381,4011,3690,18,174,25,91,434,58,2307,2307,2307,2249,2304,2308,2309,11.826352,11.096524,11.205586,11.072868,11.104997,9.868812,9.966716,9.867141,10.353587,9.875515,1.957540,1.129809,1.338445,0.719281,1.229482,1.011215,0.912334,0.881921,1.073855,1.095627,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
2,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,27226,27159,26973,26815,26489,-116,-115,-67,-186,-158,-326,70,335,300,283,260,269,128,319,291,294,310,309,-58,16,9,-11,-50,-40,2,-4,-7,-3,-2,0,-62,-129,-68,-191,-105,-281,-60,-133,-75,-194,-107,-281,2,2,-1,19,-1,-5,3193,3193,3382,3388,3389,3353,3352,12.278483,11.032454,10.455923,9.667584,10.093051,11.692048,10.701480,10.862337,11.526735,11.593877,0.586435,0.330974,-0.406414,-1.859151,-1.500825,-0.146609,-0.257424,-0.110840,-0.074366,0.000000,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
3,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,22733,22642,22512,22549,22583,-58,-128,-91,-130,37,34,44,266,245,259,247,253,34,278,237,281,211,223,10,-12,8,-22,36,30,2,10,16,18,21,21,-69,-126,-115,-140,-4,4,-67,-116,-99,-122,17,25,-1,0,0,14,-16,-21,2224,2224,2224,2224,2224,2233,2236,11.668202,10.798898,11.471852,10.962917,11.211557,12.194587,10.446281,12.446295,9.365083,9.882124,-0.526385,0.352617,-0.974443,1.597834,1.329434,0.438654,0.705234,0.797272,0.932070,0.930604,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861
4,50,3,6,1,9,Alabama,Blount County,57322,57322,57373,57711,57776,57734,57658,57673,51,338,65,-42,-76,15,183,744,710,646,618,603,133,570,592,585,589,590,50,174,118,61,29,13,5,3,19,20,28,28,-3,104,-68,-101,-119,-79,2,